# Expectation Suite
### Introduction
In order to validate your data, Great Expectations is a package that offers a battery-included set of logic to get up-and-running fast. Fully figuring out how Great Expectations works and applying it to your project, however, can be somewhat involved.

Therefore, this bootstrapped project helps you with that by making a few choices for you and offering scripts and notebooks to get you up and running fast. The choices already made here are:
- Great Expectations output will be stored on S3
- The rendered Data Docs site will be stored on S3
- You will write your own data loading logic to read data into memory as a pandas DataFrame
- You will write your own set of expectations to test the quality of this data
- The validation logic will be deployed as Docker container via AWS Lambda

To set you up for the above, the configuration file had you enter parameters that will be used throughout this project. Assuming these were properly set, you can now continue to set up your expectation suite!

### Description
This notebook can be used to generate a so-called expectation suite that can be used to run validations against your data. An expectation suite is Great Expectations jargon for a collection of expectations (or tests) that you want to run your data against. 

This notebook helps you to set up such a set of expectations, by loading a batch of data and writing expectations that can be run on it. After doing so, this set of expectations will be saved as an expectation suite.

In the last step, this expectation suite will be connected to a checkpoint, which is an object which can be called by validation logic to run new data batches against. Hence, after setting up an expectation suite and checkpoint using this notebook, the next step is to finalize the AWS lambda function found in lambda_function.py and deploy that on AWS as a Docker image. To assist with these steps Terraform configurations, a Dockerfile and a bash script ('build_image_store_on_ecr.sh) were automatically generated.

### Dependencies

#### Virtual environment
In order to run the logic contained within this notebook, make sure that it was started up from a virtual environment that contained all required Python dependencies. The easiest way to assure this is to first make a virtual environment for the project and then call initialize_project.py from within it.

To create a new virtual environment, e.g. for python 3.8, and installing the packages that can be found in the requirements.txt file, the commands are:
```sh
conda create --name <virtual_environment_name> python=3.8
conda activate <virtual_environment_name>
pip install -r </path/to/requirements.txt>
```

#### S3 buckets
For testing, Great Expectations is configured to interact with S3 buckets on AWS. To be able to run all the code in this notebook the **store_bucket** and **site_bucket** as configured in the testing_config.yml, must be provisioned.

To do so, auto-generated Terraform files can be found in the *terraform* directory of this project. To use these configurations to generate S3 buckets, open a terminal in this directory and run the following commands:

```bash
# Initialize Terraform
terraform init

# Generate deployment plan, enter yes at the prompt if the plan is correct
terraform apply
```

#### Imports and configurations
In the cell below, packages are imported and configurations are loaded from the project_config.yml file

In [1]:
# -- Imports
import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
import boto3
from supporting_functions import (TestingConfiguration,
                                  get_file_keys_from_s3,
                                  checkpoint_without_datadocs_update, 
                                  print_ge_site_link,
                                  tutorial_invoke_lambdas)
from supporting_functions import load_csv_from_s3 as load_data
from supporting_functions import load_csv_from_s3_2


import os

# -- Load parameters from configuration file
test_config = TestingConfiguration("project_config.yml")
test_config.load_config()

# -- Constant parameters
PATH_TUTORIAL_DATA = "data/"
ROW_COUNT_DELTA = .05

#### Initialization of objects
Next, objects required to interact with AWS and the Great Expectations configurations are initialized.

Specifically for Great Expectations, the `context` object is initialized. By loading the GE configuration file (found in the subdirectory great_expectations), this object stores important parameters for you to interact with GE and automatically interact with S3 buckets for storing output, pulling checkpoints and generating Data Docs sites. For more GE information, check out [their website](https://docs.greatexpectations.io/docs/)

In [2]:
# -- 1. Initialize GE and S3 objects
s3_client = boto3.client("s3")
bucket = boto3.resource("s3").Bucket(test_config.data_bucket)
context = ge.data_context.DataContext()

Invalid store configuration: Please check the configuration of your TupleS3StoreBackend named expectations_store
Invalid store configuration: Please check the configuration of your TupleS3StoreBackend named validations_store
Invalid store configuration: Please check the configuration of your TupleS3StoreBackend named checkpoint_store
/Users/jschra/opt/anaconda3/envs/grater_expectations_2/lib/python3.8/site-packages/great_expectations/datasource/data_connector/runtime_data_connector.py:133: DeprecationWarning: Specifying batch_identifiers as part of the RuntimeDataConnector config is deprecated as of v0.15.1 and will be removed by v0.18. Please configure batch_identifiers as part of Assets instead.
  warnings.warn(


#### Uploading tutorial data to S3


In [3]:
for dataset in os.listdir(PATH_TUTORIAL_DATA):
    # -- Set path to file
    path_file = os.path.abspath(PATH_TUTORIAL_DATA+dataset)

    # -- Upload to S3
    bucket.meta.client.upload_file(
        Filename=path_file, 
        Bucket=test_config.data_bucket, 
        Key=PATH_TUTORIAL_DATA+dataset
    )

S3UploadFailedError: Failed to upload /Users/jschra/GIT/Grater_expectations/github_repo/tutorial_dev/data/yellow_tripdata_2021-12.csv to ge-data-bucket-jorik/data/yellow_tripdata_2021-12.csv: An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist

#### Loading data
Now, logic must be written for loading data. The easiest way to do so is to write a new function for this that you store in supporting_functions.py, so that it can be used both in this notebook and in the lambda function.

If you have your data locally as a CSV file, such a function could be as simple as:

```python
def load_data(path: str) -> pd.DataFrame:
    df_batch = pd.read_csv(path)

    return df_batch
```

However, note that if you transfer this logic to a lambda function, data will most probably have to be downloaded from another location, for example an S3 bucket. Loading logic for such a case could look like:
def load_data()

```python
def load_csv_from_s3(
    s3_bucket_client: boto3.resource("s3").Bucket, file_key: str
) -> pd.DataFrame:
    """Function that loads a csv from S3 into a pandas DataFrame

    Parameters
    ----------
    s3_bucket_client : boto3.resource
        Instantiated s3 bucket client using boto3. Note that it should already
        be pointing to the bucket from which you want to load objects
    file_key : str
        Key to the channel within the tile to be loaded

    Returns
    -------
    pd.DataFrame
        The loaded data as pandas DataFrame
    """
    s3_object = s3_bucket_client.Object(file_key).get()
    df_batch = pd.read_csv(s3_object['Body'])

    return df_batch
```

Pay special attention to what inputs this function needs and how it knows what file to load, since this will need to be used in the lambda as well. If you are able to pass the path to the file during runtime when using a lambda (e.g. in the event sent to it), you can simply use that as a parameter of the function.


In [ ]:
# ### PUT YOUR DATA LOADING LOGIC HERE
# # To generate a testing suite, a batch of data can be used to develop
# # validations. Accordingly, in the lines below custom logic is required to load
# # a test dataset as a pandas DataFrame in the df_batch argument
# # To make the RuntimeBatchRequest complete, which is used by GE for the
# # validations, an asset_name and batch_identifier are required
# df_batch = load_data()  # Needs to be defined!
# batch_identifier = "BATCH IDENTIFIER OR LOGIC TO GENERATE IT GOES HERE"
# asset_name = "ASSET NAME OR LOGIC TO GENERATE IT GOES HERE"

# -- Get object keys from S3, sort, pick oldest
list_objects = get_file_keys_from_s3(s3_client, test_config.data_bucket, PATH_TUTORIAL_DATA)
list_objects.sort()
asset_name = list_objects[0]

# -- Load dataset
# path_batch_object = f"s3://{test_config.data_bucket}/{asset_name}"
# df_batch = load_data(path_batch_object)
# batch_identifier = "tutorial_batch_dataset"

In [ ]:
load_csv_from_s3_2(bucket, )

#### Generate batch request, generate suite and start validator
When you have succesfully written logic to load data, the loaded batch will be passed to a RuntimeBatchRequest below in order to start an expectation suite. Great Expectations requires data to be passed as a request when you want to use the context to generate things such as expectation suites and checkpoints. The RunetimeBatchRequest is used here, because we are loading data (with our own logic) at runtime and want to pass that to subsequent objects.

Using the RuntimeBatchRequest, a suite and validator are initiated and you can start writing expectations in the next cells


In [ ]:
# -- 3. Generate batch request at runtime using loaded tile
batch_request = RuntimeBatchRequest(
    datasource_name="runtime_data",
    data_connector_name="runtime_data_connector",
    data_asset_name=asset_name,
    runtime_parameters={"batch_data": df_batch},
    batch_identifiers={"batch_identifier": batch_identifier,},
)

# -- 4. Generate suite, start validator
suite = context.create_expectation_suite(
    test_config.expectations_suite_name,
    overwrite_existing=True,  
)

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=test_config.expectations_suite_name,
)

#### Expectations
Using the validator object, expectations can be formulated below. Since Great Expectations comes with many expectations out of the box, [this page](https://greatexpectations.io/expectations) is generally a good place to start browsing through these. 

Predefined expectations can be used by calling them using the validator object and passing the required arguments. For example, to run an expectation on the number of rows in a dataframe, the following snippet can be used:

```python
# Get number of rows of current batch
row_count = df_batch.shape[0]

# Make expectation where the maximum deviation from the batch number of rows is 1%
max_delta = 0.01
validator.expect_table_row_count_to_be_between(
    min_value=row_count * (1-max_delta), max_value=row_count * (1+max_delta))
```

If you want to develop custom expectations, more information can be found about there [here](https://docs.greatexpectations.io/docs/guides/expectations/creating_custom_expectations/overview)

In [ ]:
# -- Table level expectations

# -- 1. Expect columns in table
expected_columns = df_batch.columns
validator.expect_table_columns_to_match_set(
    column_set=expected_columns, exact_match=True
)

# -- 2. Expect row in between range, based on set delta
row_count = df_batch.shape[0]
validator.expect_table_row_count_to_be_between(
    min_value=row_count * (1-ROW_COUNT_DELTA), 
    max_value=row_count * (1+ROW_COUNT_DELTA),
)


In [ ]:
# -- Column level expectations

# -- 1. Values are never null
for column in expected_columns:
    validator.expect_column_values_to_not_be_null(column)

# -- 2. Date columns are parseable
date_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]
for column in date_columns:
    validator.expect_column_values_to_be_dateutil_parseable(column)

# -- 3. Check dtypes, assuming dtypes of the batch dataset are correct (this is
#       something you might rather want to hard-code for real products)
dict_dtypes = {}
for column, dtype in zip(df_batch.dtypes.index, df_batch.dtypes):
    dict_dtypes[column] = str(dtype)

for column, dtype in dict_dtypes.items():
    validator.expect_column_values_to_be_of_type(column, dtype)

# -- 4. Expect values of specific columns to be between lower- and upper bounds
dict_bounds = {"VendorID":[1,2],
              "payment_type":[1,4]
              }

for column, (lower_bound, upper_bound) in dict_bounds.items():
    # NOTE: for large datasets, this expectation is really slow. Using seperate tests
    # for what the minimum should be and the maximum should be is a lot faster, while
    # achieving the same test-wise
    validator.expect_column_values_to_be_between(column, lower_bound, upper_bound)

In [ ]:
# -- Dynamic validations
test_column = "passenger_count"
max_passenger_count = df_batch[test_column].max()

validator.expect_column_max_to_be_between(
    column=test_column,
    min_value={
        "$PARAMETER": "min_max_passenger_count",
        "$PARAMETER.min_max_passenger_count": max_passenger_count
        },
    max_value={
        "$PARAMETER": "max_max_passenger_count",
        "$PARAMETER.max_max_passenger_count": max_passenger_count+2
        },
)

#### Finalize suite and create checkpoint
After running all the expectations that you want to apply to the data, the cell below can be executed to save the set of expectations as a suite and couple it with a checkpoint. This checkpoint can be used in other scripts (lambda_function.py) by passing a new batch of data (as RuntimeBatchRequest) along with the checkpoint name to the `run_checkpoint` method of an initialized DataContext (which is the same as the `context` object initialized in this notebook. Such a call would look like:

```python
results = context.run_checkpoint(
    checkpoint_name="CONFIG_NAME",
    validations=[{"batch_request": batch_request}],
    )
```

**NOTE**: you should choose what kind of checkpoint you want to use below: one with or without automatic Data Docs updates

When the code below is ran, Great Expectations automatically saves the expectation suite and checkpoint to S3 via the validator and context objects.

In [ ]:
# -- 6. Save suite
validator.save_expectation_suite(discard_failed_expectations=False)

# -- 7. Create checkpoint
#       Two options are provided here for creating a checkpoint: 
#       1. Use a SimpleCheckpoint, which contains an action to automatically update the
#          Data Docs website whenever a validation is run
#       2. Use checkpoint_without_datadocs_update, which generates the same
#          SimpleCheckpoint but without an automatic Data Docs update action. This is
#          useful if you run many validations (1000+) in parallel, since rendering the
#          Data Docs website becomes very slow in that case          

# -- 7.1 Create Simple checkpoint with automatic data docs updates (DEFAULT)
checkpoint_config = {
    "name": test_config.checkpoint_name,
    "config_version": 3,
    "class_name": "SimpleCheckpoint",
    "expectation_suite_name": test_config.expectations_suite_name,
    "run_name_template": test_config.run_name_template,
}

# -- 7.2 Create checkpoint without automatic data docs update
# checkpoint_config = checkpoint_without_datadocs_update(test_config)

context.add_checkpoint(**checkpoint_config)

#### Instantiate Data Docs website
Next, the command below can be ran to build the Data Docs website on S3, which provides an interactive user interface in which you can browse through expectation suites and check validation results. More on Data Docs can be found [here](https://docs.greatexpectations.io/docs/reference/data_docs/)

In general, if you generate the website once, new validations should automatically be loaded onto it. If the website start lagging behind, just rerun the command below.

In [ ]:
ge_site_output = context.build_data_docs()
print_ge_site_link(ge_site_output)

#### Invoke Lambdas

In [ ]:
# -- Initialize client
lambda_client = boto3.client('lambda', region_name='eu-west-1')

# -- Get object keys from S3, sort, pick oldest
list_objects_lambda = list_objects[1:]

# -- Invoke lambdas
responses = tutorial_invoke_lambdas(lambda_client, list_objects_lambda)

In [ ]:
print_ge_site_link(ge_site_output)

#### Stop Jupyter server
After running all commands above and generating an expectation suite, checkpoint and website, run the command below to stop the Jupyter server from running

In [ ]:
os.system("jupyter notebook stop")